# Random Forest and Gradient Boosing Models

In [2]:
import pathlib

import feather
import pandas as pd
import numpy as np

# Scikitlearn

In [3]:
DATA_PATH = pathlib.Path('./data')
RESULTS_PATH = pathlib.Path('./results')

In [5]:
X_train = feather.read_dataframe(DATA_PATH / 'X_train')
y_train = feather.read_dataframe(DATA_PATH / 'y_train')

X_test = feather.read_dataframe(DATA_PATH / 'X_test')
y_test = feather.read_dataframe(DATA_PATH / 'y_test')

In [6]:
X_train

,addr_state,annual_inc,application_type,dti,emp_length,fico_range_high,grade,home_ownership,initial_list_status,installment,...,pub_rec,pub_rec_bankruptcies,purpose,revol_bal,revol_util,sub_grade,term,title,total_acc,verification_status
0,38,-0.241008,0,-1.524413,10,-0.250,2,1,1,-1.422362,...,0.0,0.0,2,-1.238791,-0.936480,13,0,9171,-0.956582,0
1,41,0.000000,0,-0.148651,10,0.750,2,1,1,0.956811,...,0.0,0.0,11,0.520866,-1.580984,10,0,2386,0.733641,0
2,14,-0.045088,1,-0.712392,10,0.250,1,1,1,0.153068,...,0.0,0.0,4,-0.340926,0.021687,8,1,9171,-0.452765,0
3,38,0.684069,0,0.514275,3,0.250,5,1,1,-0.349422,...,0.0,0.0,6,0.539029,0.230316,25,1,16398,0.601586,1
4,10,-0.934900,0,-0.789250,4,0.125,2,3,1,0.070675,...,0.0,0.0,2,-0.242778,0.319369,12,0,9171,-2.100133,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
733446,4,0.387024,0,0.286147,10,1.125,1,1,1,-0.144976,...,0.0,0.0,1,-0.233197,-0.019230,5,1,29772,0.244863,2
733447,31,-0.227952,0,0.401065,1,-0.500,3,3,0,0.678864,...,0.0,0.0,2,0.096805,0.596765,17,0,26508,0.733641,2
733448,43,1.281420,0,0.524057,10,-0.250,1,3,0,0.489652,...,0.0,0.0,2,-0.588748,0.064165,7,0,7489,-0.368141,2
733449,4,1.621499,0,-0.363949,9,0.500,1,1,0,1.264897,...,0.0,0.0,1,0.859762,0.555453,7,0,30463,0.000000,2


# Outline

- What is a random forest model? 
- How does it work? 
- Why does it work well?
- How to use it in scikit-learn?
- Train and compare on holdout set

## Extra (Not part of presentation)
- What is a gradient boosting model? 
- How does it work? 
- Why does it work well?
- How to use it in scikit-learn?
- Train and compare on holdout set
